In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
import mlflow
import mlflow.sklearn
from sklearn.metrics import roc_auc_score, fbeta_score
from sklearn.model_selection import train_test_split

try:
    path = ''
    train = pd.read_csv(path + "train_encoded.csv")
    test = pd.read_csv(path + "test_encoded.csv")
    train_old = pd.read_csv(path + 'train_sample.csv')

except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check path. Error: %s", e
    )

## The predicted column is "TARGET" which is a scalar Binary [0, 1]
train = train.merge(train_old[['SK_ID_CURR', 'TARGET']], on='SK_ID_CURR', how='left')
test['TARGET'] = np.nan
train_y = train['TARGET']
test_y = test['TARGET']
del train_old
train = train.dropna(subset=['TARGET'])
test = test.dropna(subset=['TARGET'])
train = train.drop(columns=["TARGET"], axis=1)
test = test.drop(columns=["TARGET"], axis=1)
train_y.dropna(inplace=True)
test_y.dropna(inplace=True)
train.fillna(train.median(), inplace=True)
test.fillna(test.median(), inplace=True)
indices=train['SK_ID_CURR']
test = test.set_index('SK_ID_CURR')
train = train.set_index('SK_ID_CURR')

# split data into train and test sets
X_train, X_test, y_train, y_test = train, test, train_y, test_y #train_test_split(X, y, test_size=0.2, random_state=42)

# set the hyperparameters for the XGBoost model
hyperparameters = {
    "learning_rate": 0.1,
    "n_estimators": 100,
    "max_depth": 3
}

# start a new MLFlow run
with mlflow.start_run() as run:
    # log the hyperparameters
    mlflow.log_params(hyperparameters)

    # create the XGBoost model
    model = xgb.XGBClassifier(**hyperparameters)
    model.fit(X_train, y_train)

    # make predictions on the test set
    y_pred = model.predict(X_test)

    # calculate the AUC score
    auc = roc_auc_score(y_test, y_pred)
    print("AUC score:", auc)
    mlflow.log_metric("AUC", auc)

    # calculate the FBeta score
    fbeta = fbeta_score(y_test, y_pred, beta=0.5)
    print("FBeta score (beta=0.5):", fbeta)
    mlflow.log_metric("FBeta (beta=0.5)", fbeta)

    # log the model to the run
    mlflow.sklearn.log_model(model, "model")

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [15]:
test_y

Series([], Name: TARGET, dtype: float64)